In [3]:
class RawData:
    def __init__(
        self,
        env_name,
        brain_name,
        search_seed,
        env_seed,
        env_sdf,
        goal,
        robot_urdf,
        score,
    ):
        self.env_name = env_name
        self.brain_name = brain_name
        self.search_seed = search_seed
        self.env_seed = env_seed
        self.env_sdf = env_sdf
        self.goal = goal
        self.robot_urdf = robot_urdf
        self.score = score

    def __str__(self):
        return (
            f"Data(\n"
            f"  env_name={self.env_name},\n"
            f"  brain_name={self.brain_name},\n"
            f"  search_seed={self.search_seed},\n"
            f"  env_seed={self.env_seed},\n"
            f"  env_sdf={self.env_sdf[0:50]},\n"
            f"  goal={self.goal},\n"
            f"  robot_urdf={self.robot_urdf[0:50]},\n"
            f"  score={self.score}\n"
            ")"
        )

# read Raw Data

In [2]:
import os
import re

data_dir = "../final/data"
env_pattern = r"terrain\d+"
seed_list = ["seed1234", "seed2345"] 
brain_type = "ea"

fitness_file = "best_fitness.txt"
body_file = "body.urdf"
brain_file = "brain-100.nndf"
world_file = "world.sdf"
            

In [3]:
def read_file(file_path):
    with open(file_path,"r") as f:
        file_str = f.read()
    return file_str

In [4]:
raw_data_dirs = []
raw_data_list = []
for e in os.listdir(data_dir):
    env_path = os.path.join(data_dir,e)
    match = re.match(r"[A-Za-z]+(\d+)", e)
    if match:
        env_seed = int(match.group(1))
    if os.path.isdir(env_path) and re.match(env_pattern,e):
        for s in os.listdir(env_path):
            seed_path = os.path.join(env_path,s)
            if os.path.isdir(seed_path) and (s in seed_list):
                for b in os.listdir(seed_path):
                    if re.match(r"body\d+",b):
                        body_path = os.path.join(seed_path,b)
                        raw_data_dirs.append(body_path)
                        raw_data = RawData(
                            env_name=e,
                            brain_name=brain_type,
                            search_seed=s,
                            env_seed=env_seed,
                            env_sdf=read_file(os.path.join(env_path, world_file)),
                            goal=(100, 0, 0),
                            robot_urdf=read_file(os.path.join(body_path, body_file)),
                            score=(float(read_file(os.path.join(body_path, fitness_file))), 0 , 0),
                        )
                        raw_data_list.append(raw_data)

In [9]:
str(raw_data_list[120])

'Data(\n  env_name=terrain36199,\n  brain_name=ea,\n  search_seed=seed2345,\n  env_seed=36199,\n  env_sdf=<sdf>\n    <model name="terrain0">\n        <pose>1.,\n  goal=(100, 0, 0),\n  robot_urdf=<robot name = "robot">\n    <link name="link1">\n   ,\n  score=(0.6332962657929321, 0, 0)\n)'

# save data

In [8]:
from datetime import datetime
import pickle

now = datetime.now()
formatted_now = now.strftime("%y%m%d%H%M")
with open(f"./data/rawdata_{formatted_now}.pkl", "wb") as file:
    pickle.dump(raw_data_list, file)

# read data

In [4]:
import pickle
def read_data(file_path):
    with open(file_path, "rb") as f:
        loaded_data = pickle.load(f)
    return loaded_data

In [5]:
raw_data= read_data("./data/rawdata_2304040613.pkl")

In [6]:
str(raw_data[0])

'Data(\n  env_name=terrain36199,\n  brain_name=ea,\n  search_seed=seed1234,\n  env_seed=36199,\n  env_sdf=<sdf>\n    <model name="terrain0">\n        <pose>1.,\n  goal=(100, 0, 0),\n  robot_urdf=<robot name = "robot">\n    <link name="link1">\n   ,\n  score=(0.24942633879748496, 0, 0)\n)'

# transform data

In [1]:
def convert_urdf_to_sequence(urdf_string):
    root = ET.fromstring(urdf_string)
    # Extract the desired information
    sequence = []
    for element in root:
        if element.tag == "joint":
            parent_name = element.find("parent").attrib["link"]
            node_name = element.find("child").attrib["link"]
            joint_axis = element.find("axis").attrib["xyz"]
            joint_type = element.attrib["type"]
            sequence.append((parent_name, node_name, joint_axis, joint_type))
        elif element.tag == "link":
            node_name = element.attrib["name"]
            # Extract the link type and size from the visual geometry
            visual_geometry = element.find("visual/geometry")
            link_type = None
            link_size = None
            if visual_geometry is not None:
                link_type = list(visual_geometry)[0].tag
                if link_type == "sphere":
                    radius = float(visual_geometry.find("sphere").attrib["radius"])
                    link_size = (radius,radius,radius)
                elif link_type == "box":
                    size_str = visual_geometry.find("box").attrib["size"]
                    link_size = tuple([float(s) for s in size_str.split()])
                elif link_type == "cylinder":
                    length = float(visual_geometry.find("cylinder").attrib["length"])
                    radius = float(visual_geometry.find("cylinder").attrib["radius"])
                    link_size = (radius,radius,length)

            # Extract the sensor tag (assuming it's stored in the material name attribute)
            sensor_color = element.find("visual/material").attrib["name"] if element.find("visual/material") is not None else "unknown"
            sensor_tag = True if "sensored" in sensor_color else False
            sequence.append((node_name, link_type, link_size, sensor_tag))
    return sequence

In [2]:
def convert_sdf_to_gridmap(sdf_string, return_submap=False):
    # Parse the SDF string
    root = ET.fromstring(sdf_string)
    # Create a grid map representation
    grid_resolution = 1  # Adjust as needed
    grid_size = [10, 10]  # Adjust as needed (this should be large enough to cover the entire space)
    grid_map = np.zeros(grid_size, dtype=np.double) #grid_map = np.zeros(grid_size, dtype=np.uint8)
    # Iterate over each model in the SDF file
    for model in root.findall(".//model"):
        # Extract box dimensions and pose
        box_size = model.find(".//box/size").text.split()
        box_size = [float(dim) for dim in box_size]
        box_pose = model.find(".//pose").text.split()
        box_position = [float(coord) for coord in box_pose[:3]]
        box_rotation = [float(angle) for angle in box_pose[3:]]
        # Fill in the grid cells corresponding to the box
        offset = [int(coord / grid_resolution + size / 2) for (coord, size) in zip(box_position[0:2],grid_size)]
        for i in range(offset[0], min(grid_size[0], offset[0] + int(box_size[0] / grid_resolution))):
            for j in range(offset[1], min(grid_size[1], offset[1] + int(box_size[1] / grid_resolution))):
                grid_map[i, j] = box_size[2] #1
    if return_submap:
        return grid_map[6:10,3:7]
    else:
        return grid_map

In [5]:
import numpy as np
import xml.etree.ElementTree as ET

from src.utils.data_utils import PreprocessedData as PreprocessedData
from src.utils.data_utils import read_data as read_data
from src.utils.data_utils import save_data as save_data

class DataTransformer:
    def __init__(self):
        pass
    
    def generate_sequence(self, data_file, save_dir="./data/preprocessed", subset=None):   
        raw_data = read_data(data_file)
        processed_data = []
        for single_data in raw_data[0:subset]:
            sdf_sequence = convert_sdf_to_gridmap(single_data.env_sdf, return_submap=True)
            goal_sequence = single_data.goal
            urdf_sequence = convert_urdf_to_sequence(single_data.robot_urdf)
            score_sequence = single_data.score
            processed_data.append(PreprocessedData(sdf_sequence, goal_sequence, urdf_sequence, score_sequence))
        
        file_path = save_data(save_dir, processed_data)
        return file_path
        

In [7]:
data_tf = DataTransformer()
prepcossed_file = data_tf.generate_sequence("./data/raw/rawdata_2304041648.pkl",subset=10)

In [14]:
preprocessed_data = read_data(prepcossed_file)

# test

In [1]:
from src.utils.data_utils import read_data as read_data
preprocess_file = "./data/preprocessed/data_2304041728.pkl"
preprocessed_data = read_data(preprocess_file)

### convert dataset to alpaca sequence

In [4]:
import pandas as pd
def construct_alpaca_data(preprocessed_data):
    df = pd.DataFrame(columns=["instruction", "input_sequence", "output_sequence"])
    for single_data in preprocessed_data:
        sdf_sequence, urdf_sequence, goal_sequence, score_sequence =single_data.round_sequence(2)
        instruction = "Based on the following gridmap and goal, give me the robot description and its score"
        input_sequence = f"Grid map is {sdf_sequence}, the goal position is {goal_sequence}."
        output_sequence = f"Robot description is {urdf_sequence}, its final position is {score_sequence}"
        row_data = {"instruction": instruction, "input_sequence": input_sequence, "output_sequence":output_sequence }
        df = df.append(row_data, ignore_index=True)
    return df

In [5]:
# alpaca_df = construct_alpaca_data(preprocessed_data)

In [6]:
# from datetime import datetime
# now = datetime.now()
# formatted_now = now.strftime("%y%m%d%H%M")
# file_path = f"./data/alpacaed/data_{formatted_now}.json"
# alpaca_df.to_json(file_path, orient="records", indent=4)

### step1 Dataset

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

tokenizer_path = "./data/tutorial/tokenizer"
trainer_output_dir = "./data/tutorial/model"
trainer_save_dir = "./data/tutorial/finetuned_model"
preprocess_file = "./data/preprocessed/data_2304041728.pkl"

In [2]:
from src.utils.data_utils import read_data as read_data
preprocessed_data = read_data(preprocess_file)
dataset = [seq.round_str(2) for seq in preprocessed_data]

### step2 hugging face fine tuned

In [3]:
import numpy as np
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]
        encoding = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=self.max_length)
        return {key: tensor.squeeze(0) for key, tensor in encoding.items()}

In [10]:
from transformers import GPT2LMHeadModel, GPT2Config
from transformers import GPT2Tokenizer

def load_gpt_model(pretrained_model_name_or_path):
    config = GPT2Config.from_pretrained(pretrained_model_name_or_path)
    model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path, config=config)
    return model

def load_gpt_tokenizer(pretrained_model_name_or_path):
    tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path)
    pad_token_exists = tokenizer.pad_token is not None
    if not pad_token_exists:
        tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

In [10]:
from transformers import GPT2LMHeadModel, GPT2Config, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load pre-trained GPT model and tokenizer
model_name_or_path = "gpt2"
tokenizer = load_gpt_tokenizer(model_name_or_path)
model = load_gpt_model(model_name_or_path)
config = GPT2Config.from_pretrained(model_name_or_path)

# initialize torch dataset
train_dataset = CustomDataset(data=dataset[0:10], tokenizer=tokenizer, max_length=128)
    
# Fine-tune the model using your dataset
training_args = TrainingArguments(
    output_dir=trainer_output_dir,
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=train_dataset,
)

trainer.train()
trainer.save_model(trainer_save_dir)

Using pad_token, but it is not set yet.


Step,Training Loss


TrainOutput(global_step=9, training_loss=1.7774594624837239, metrics={'train_runtime': 0.8334, 'train_samples_per_second': 35.995, 'train_steps_per_second': 10.799, 'total_flos': 1959690240000.0, 'train_loss': 1.7774594624837239, 'epoch': 3.0})

### load model

In [5]:
# src/model/gpt_model.py

from transformers import GPT2LMHeadModel, GPT2Config

def load_fine_tuned_gpt_model(model_dir):
    config = GPT2Config.from_pretrained(model_dir)
    model = GPT2LMHeadModel.from_pretrained(model_dir, config=config)
    return model

In [6]:
loaded_model = load_fine_tuned_gpt_model(trainer_save_dir)

In [11]:
tokenizer = load_gpt_tokenizer("gpt2")

Using pad_token, but it is not set yet.


In [12]:
dataset[0]

"grid map: [[0.11 0.18 0.18 0.02]\n [0.15 0.05 0.01 0.17]\n [0.19 0.17 0.14 0.04]\n [0.05 0.04 0.18 0.13]], goal: (100, 0, 0), urdf: [('link1', 'sphere', (0.08, 0.08, 0.08), False), ('link2', 'cylinder', (0.1, 0.1, 0.22), True), ('link3', 'cylinder', (0.09, 0.09, 0.36), True), ('link1', 'link2', '0 1 0', 'spherical'), ('link2', 'link3', '0 1 0', 'revolute')], score: (0.25, 0, 0)"

In [13]:
prompt = "grid map: [[0.11 0.18 0.18 0.02]\n [0.15 0.05 0.01 0.17]\n [0.19 0.17 0.14 0.04]\n [0.05 0.04 0.18 0.13]], goal: (100, 0, 0),"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

In [15]:
max_length = 128
num_return_sequences = 1
output_sequences = loaded_model.generate(
    input_ids=input_ids,
    max_length=max_length,
    num_return_sequences=num_return_sequences,
    no_repeat_ngram_size=2,
    do_sample=True,
    top_p=0.95,
    top_k=50,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [16]:
generated_sequences = []
for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    generated_sequences.append(text)

In [17]:
generated_sequences

['grid map: [[0.11 0.18 0.18 0.02]\n [0.15 0.05 0.01 0.17]\n [0.19 0.17 0.14 0.04]\n [0.05 0.04 0.18 0.13]], goal: (100, 0, 0), [(0, (0)], (1, 1), (2, 2), ((1 + (goal-1) * 2) - goal-0), 0);\nThe last three values to examine are the (from 0 to 100) and the top five.']

In [ ]:
# src/model/inference.py

def generate_text(prompt, model_dir, max_length=128, num_return_sequences=1):
    tokenizer = load_gpt_tokenizer(model_dir)
    model = load_fine_tuned_gpt_model(model_dir)

    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        do_sample=True,
        top_p=0.95,
        top_k=50,
    )

    generated_sequences = []
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        generated_sequences.append(text)

    return generated_sequences

### evaluate

In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [1]:
import src.model.inference as inference

In [2]:
import src.data_preprocessing.data_transformer as data_transformer
preprossed_file = "./data/preprocessed/data_2304041728.pkl"
dataset = data_transformer.preprocess_data(preprossed_file)

In [35]:
dataset[0]

"grid map: [[0.11 0.18 0.18 0.02]\n [0.15 0.05 0.01 0.17]\n [0.19 0.17 0.14 0.04]\n [0.05 0.04 0.18 0.13]], goal: (100, 0, 0), urdf: [('link1', 'sphere', (0.08, 0.08, 0.08), False), ('link2', 'cylinder', (0.1, 0.1, 0.22), True), ('link3', 'cylinder', (0.09, 0.09, 0.36), True), ('link1', 'link2', '0 1 0', 'spherical'), ('link2', 'link3', '0 1 0', 'revolute')], score: (0.25, 0, 0)"

In [29]:
prompt = "grid map: [[0.11 0.18 0.18 0.02]\n [0.15 0.05 0.01 0.17]\n [0.19 0.17 0.14 0.04]\n [0.05 0.04 0.18 0.13]], goal: (100, 0, 0),"
# prompt = "grid map: [[0.09 0.14 0.13 0.06]\n [0.08 0.02 0.02 0.14]\n [0.19 0.17 0.01 0.16]\n [0.01 0.1  0.   0.13]], goal: (100, 0, 0),"
tokenizer_name_or_path = "gpt2"
model_dir = "./data/tutorial/finetuned_model"
model_dir = "./data/tutorial/finetuned_model_2304050545"
model_dir = "./data/tutorial/finetuned_model_2304051426"
max_length = 512 #256

In [30]:
infered_sequence = inference.generate_text(prompt, tokenizer_name_or_path, model_dir, max_length, num_return_sequences=1)

Using pad_token, but it is not set yet.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
print(infered_sequence[0][123:])

urdf: [('link1','sphere', (0,0), (1,1), True), ('link2', map ( 0 0 1, 1), False),('spherical')], score: (-0(0./0/0) errilla False)], map('cylinder', (-1.1,- 0., 0) True) ('cylcylinders', ('spike', ((0.,0.-0.", 0.",0.) True))],score: ('0.(0.. 0.) ile True, (-2 0 for 0…)], scored: ((1.. (- 0.. -0)) False, ('box', ("0 0   0", False); ('fixed', (−0\.18, -18%, 0%, 'fixed')}, score:-0.: (2, 2, True)}], iles: ([0.08, −.08%, False)+ ('comment', [ 'box'] ( 'cylind 0.19, null), [spacing: 0.<spaces 0.</spacer>.
 (after 0 second, 'link3', False)} (link4', '[0 1 0', '(0', 0 part),'revolute'), ('links5',['cyl',('box1 0 FT', ['spoiler', ([('revure',0 '0  0 regulator', function '1'),''links2'), '[1 1', True', () 'linked3'), '(link51 ', "spider', ({0.[0.]0], '2  '3 0 OFF', ())], result: '(1.13, ('1 False', (), ('revured'), (fixed), ['link6', '.1 CO 0 '',', '/spool', (!0.? 0.? 1, 'true'), ([link7', ', '4 01 ', False'), ($0._link8', "\cylid', (_link9', "_box01 100 0′', "'spark', (/0? 0? 1,- 1)), True'), 

In [1]:
robot_description = [('link1', 'box', (0.3, 0.3, 0.32), False), 
     ('link2', 'box', (0.2, 0.13, 0.32), True), 
     ('link3', 'box', (0.25, 0.39, 0.13), True), 
     ('link1', 'link2', '0 0 1', 'revolute'), 
     ('link2', 'link3', '0 1 0', 'revolute')] 
score = 0.38
